In [ ]:
#connecting to google drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#installing required packages
!pip install streamlit pandas matplotlib seaborn plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.2 MB/s eta 0:00:00


In [ ]:
#loading dataset
import seaborn as sns
import pandas as pd
import numpy as np
df=pd.read_csv("/content/drive/MyDrive/Data/imdb-movies-dataset.csv")
df

,Poster,Title,Year,Certificate,Duration (min),Genre,Rating,Metascore,Director,Cast,Votes,Description,Review Count,Review Title,Review
0,https://m.media-amazon.com/images/M/MV5BYWRkZj...,The Idea of You,2023.0,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","28,744","Solène, a 40-year-old single mom, begins an un...",166,Hypocrisy as an idea,"This film, as well as the reaction to it, is a..."
1,https://m.media-amazon.com/images/M/MV5BZGI4NT...,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","22,248","Many years after the reign of Caesar, a young ...",183,A phenomenal start to another trilogy!,"I'm a big fan of all the planet of the apes, a..."
2,https://m.media-amazon.com/images/M/MV5BZjIyOT...,Unfrosted,2023.0,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...","18,401","In 1963 Michigan, business rivals Kellogg's an...",333,not funny,Pretty much the worst criticism you can lay on...
3,https://m.media-amazon.com/images/M/MV5BMjA5Zj...,The Fall Guy,2023.0,PG-13,126.0,"Action, Comedy, Drama",7.3,73.0,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...","38,953",A down-and-out stuntman must find the missing ...,384,Everything you needed and more!,Just got out of the Austin premier at SXSW and...
4,https://m.media-amazon.com/images/M/MV5BNTk1MT...,Challengers,2023.0,R,131.0,"Drama, Romance, Sport",7.7,82.0,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...","32,517","Tashi, a former tennis prodigy turned coach, t...",194,"Watch ""Match Point"" instead",This is a tough one. I liked the concept and t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,https://m.media-amazon.com/images/M/MV5BMzg5MW...,The Greatest Show on Earth,2020.0,U,152.0,"Drama, Family, Romance",6.5,76.0,Cecil B. DeMille,"James Stewart, Charlton Heston, Betty Hutton, ...","16,078","The dramatic lives of trapeze artists, a clown...",128,"Hey, doesn't anyone remember Last Emperor?",It constantly amazes me that people carp that ...
9996,https://m.media-amazon.com/images/M/MV5BYzA0ZG...,Berserk: Ougon Jidai-hen I - Haou no Tamago,2020.0,NaN,76.0,"Animation, Action, Adventure",7.5,NaN,Toshiyuki Kubooka,"Hiroaki Iwanaga, Carrie Keranen, Takahiro Saku...","14,300",A lone sellsword named Guts gets recruited int...,12,Masterfully directed climatic epic saga,Few stories can capture your mind and soul in ...
9997,https://m.media-amazon.com/images/M/MV5BM2U1Mj...,Is-slottet,2020.0,NaN,78.0,"Mystery, Drama",6.5,NaN,Per Blom,"Line Storesund, Hilde Nyeggen Martinsen, Meret...",740,A couple of twelve-year-old Norwegian girls st...,4,Beautiful Film,"This film might not be to everyone's taste, it..."
9998,https://m.media-amazon.com/images/M/MV5BMTAwOD...,Loving Pablo,2020.0,A,123.0,"Biography, Crime, Drama",6.4,42.0,Fernando León de Aranoa,"Javier Bardem, Penélope Cruz, Peter Sarsgaard,...","22,447",A journalist strikes up a romantic relationshi...,84,That film should be in Spanish,Why anyone (the director?) made Spanish actors...


In [ ]:
#data preprocessing
df.dropna(inplace=True)

In [ ]:
# Streamlit UI Code
import streamlit as st
import pandas as pd

st.title("🎬 IMDb Movies Dashboard")
#slider
genre = df['Genre'].dropna().str.split(',').explode().str.strip().unique()
selected_genre = st.sidebar.selectbox("Select Genre", sorted(genre))

rating_min, rating_max = float(df['Rating'].min()), float(df['Rating'].max())
rating_range = st.sidebar.slider("IMDb Rating Range", rating_min, rating_max, (6.0, 9.0))

df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
year_min, year_max = int(df['Year'].min()), int(df['Year'].max())
year_range = st.sidebar.slider(" Release Year Range", year_min, year_max, (2000, 2023))

df['Votes'] = df['Votes'].astype(str).str.replace(',', '').astype(float)

# Checkbox
popular_only = st.sidebar.checkbox("Only Popular Movies (10k+ votes)", value=False)

#Filter Data
filtered_df = df[
    df['Genre'].str.contains(selected_genre, na=False) &
    df['Rating'].between(*rating_range) &
    df['Year'].between(*year_range)
]

if popular_only:
    filtered_df = filtered_df[filtered_df['Votes'] > 10000]

st.subheader(f"Showing {len(filtered_df)} movies")
st.dataframe(filtered_df[['Title', 'Genre', 'Year', 'Rating', 'Votes']].reset_index(drop=True))

In [ ]:
#visualisations
import plotly.express as px
import numpy as np

# 1. Bar Chart
avg_rating_per_genre = (
    filtered_df['Genre'].dropna()
    .str.split(',')
    .explode()
    .str.strip()
    .reset_index()
    .join(filtered_df[['Rating']], on='index')
    .groupby('Genre')['Rating']
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

st.plotly_chart(
    px.bar(
        avg_rating_per_genre,
        x=avg_rating_per_genre.index,
        y=avg_rating_per_genre.values,
        labels={'x': 'Genre', 'y': 'Average Rating'},
        title="⭐ Average Rating per Genre"
    )
)

# 2. Line Chart
movies_per_year = filtered_df.groupby('Year').size().reset_index(name='Movie Count')

st.plotly_chart(
    px.line(
        movies_per_year,
        x='Year',
        y='Movie Count',
        title="🎬 Number of Movies Released per Year"
    )
)

# 3. Pie Chart
genre_counts = (
    filtered_df['Genre'].dropna()
    .str.split(',')
    .explode()
    .str.strip()
    .value_counts()
    .head(5)
)

st.plotly_chart(
    px.pie(
        names=genre_counts.index,
        values=genre_counts.values,
        title="🎭 Genre Distribution (Top 5)"
    )
)

In [ ]:
#  Summary Statistics
st.subheader("📈 Summary Statistics")

st.markdown(f" Number of Movies:** {len(filtered_df)}")
st.markdown(f" Average Rating:** {round(filtered_df['Rating'].mean(), 2)}")
st.markdown(f" Median Rating:** {round(filtered_df['Rating'].median(), 2)}")
st.markdown(f" Average Votes:** {int(filtered_df['Votes'].mean())}")
